In [1]:
import functools
from functools import reduce  
import numpy as np
import tensorflow as tf

In [5]:
class test_1(object):
    def __init__(self,
                batch_size = 128,
                learning_rate = 0.001,
                training_epoch = 10,
                display_step = 5,
                layer_units_num = 100):
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.training_epoch = training_epoch
        self.display_step = display_step
        self.layer_units_num = layer_units_num
        
    def dense_to_one_hot(self,labels_dense):
        """标签 转换one hot 编码
        输入labels_dense 必须为非负数
        2016-11-21
        """
        num_classes = len(np.unique(labels_dense)) # np.unique 去掉重复函数
        raws_labels = labels_dense.shape[0]
        index_offset = np.arange(raws_labels) * num_classes
        labels_one_hot = np.zeros((raws_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
        return labels_one_hot  
        
    def Preprocessing(self, trainX, trainY, seed=False):
        trainY = self.dense_to_one_hot(trainY)
        self.in_length= in_length= trainX.shape[1]
        self.in_width= in_width= trainX.shape[2]
        self.out_classes= out_classes= trainY.shape[1]
        
        if seed:
            tf.set_random_seed(20170204)
        weights = {
            'out': tf.Variable(tf.truncated_normal(shape=[self.layer_units_num, out_classes], 
                                                mean=0., stddev=1., seed=None, dtype=tf.float32),
                               trainable=True, name='Weight_full_out')
        }
        
        biases = {
            'out': tf.Variable(tf.truncated_normal([out_classes]), trainable=True, name= 'Biases_full_out')
        }    
        
        self.weights = weights
        self.biases = biases        
        
        X = tf.placeholder(dtype=tf.float32, shape=[None, in_length, in_width], name='trainX') # 批次，时间序列，多因子
        Y = tf.placeholder(dtype= tf.float32, shape=[None, out_classes], name='trainY') 
        keep_prob = tf.placeholder(dtype= tf.float32)
        self.X = X
        self.Y = Y
        self.keep_prob = keep_prob  
        
    def Network(self):
        
        with tf.name_scope('layer_1'):            
            monolayer_1 = tf.nn.rnn_cell.BasicLSTMCell(num_units= self.layer_units_num, 
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)            
            monolayer_1 = tf.nn.rnn_cell.DropoutWrapper(cell=monolayer_1, output_keep_prob= keep_prob)

        
        with tf.name_scope('layer_2'):        
            monolayer_2 = tf.nn.rnn_cell.BasicLSTMCell(num_units= self.layer_units_num,
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)
            monolayer_2 = tf.nn.rnn_cell.DropoutWrapper(cell=monolayer_2, output_keep_prob= keep_prob)
        
        with tf.name_scope('layer_3'):                  
            monolayer_3 = tf.nn.rnn_cell.BasicLSTMCell(num_units= self.layer_units_num,
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)
            monolayer_3 = tf.nn.rnn_cell.DropoutWrapper(cell=monolayer_3, output_keep_prob= keep_prob)
        
        with tf.name_scope('layer_Final'):
            monolayer_final = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.layer_units_num, 
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)
        
        with tf.name_scope('Layer_Structure_Combination'):
            layer_units_num = self.layer_units_num
            Layers = tf.nn.rnn_cell.MultiRNNCell(cells=[monolayer_1,monolayer_2,monolayer_3,monolayer_final],
                                                state_is_tuple = True)    
        self.Layers = Layers
        return Layers
        
    def Model(self):
        X = self.X
        keep_prob = self.keep_prob
        
        X = tf.transpose(X, [1, 0, 2])
        X = tf.reshape(X, [-1,self.in_width])
        X = tf.split(split_dim=0, num_split=self.in_length, value=X)
        
        with tf.name_scope('layer_1'):            
            monolayer_1 = tf.nn.rnn_cell.BasicLSTMCell(num_units= self.layer_units_num, 
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)            
            monolayer_1 = tf.nn.rnn_cell.DropoutWrapper(cell=monolayer_1, output_keep_prob= keep_prob)

        
        with tf.name_scope('layer_2'):        
            monolayer_2 = tf.nn.rnn_cell.BasicLSTMCell(num_units= self.layer_units_num,
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)
            monolayer_2 = tf.nn.rnn_cell.DropoutWrapper(cell=monolayer_2, output_keep_prob= keep_prob)
        
        with tf.name_scope('layer_3'):                  
            monolayer_3 = tf.nn.rnn_cell.BasicLSTMCell(num_units= self.layer_units_num,
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)
            monolayer_3 = tf.nn.rnn_cell.DropoutWrapper(cell=monolayer_3, output_keep_prob= keep_prob)
        
        with tf.name_scope('layer_Final'):
            monolayer_final = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.layer_units_num, 
                                                       forget_bias=1., state_is_tuple=True, activation=tf.tanh)
        
        with tf.name_scope('Layer_Structure_Combination'):
            layer_units_num = self.layer_units_num
            Layers = tf.nn.rnn_cell.MultiRNNCell(cells=[monolayer_1,monolayer_2,monolayer_3,monolayer_final],
                                                state_is_tuple = True)    
        
        outputs,_ = tf.nn.rnn(cell=monolayer_final, inputs=X, dtype=tf.float32)
        output = outputs[-1]

        return tf.nn.bias_add(value= tf.matmul(output, self.weights['out']), bias= self.biases['out'])  
        
    def train(self, trainX, trainY, seed=False):
        self.sess = tf.InteractiveSession()
        
        self.Preprocessing(trainX, trainY, seed)
        
        tmp = self.Model()
        
        self.predict = tf.nn.softmax(tmp)
        
        self.cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(tmp, self.Y))
        
        optimizer = tf.train.AdamOptimizer(learning_rate= self.learning_rate) # 0 设置训练器
        grads_and_vars = optimizer.compute_gradients(self.cost)
        for i, (grid, var) in enumerate(grads_and_vars):
            if grid != None:
                grid = tf.clip_by_value(grid, -1., 1.)
                grads_and_vars[i] = (grid, var)
        optimizer = optimizer.apply_gradients(grads_and_vars)
        self.optimizer = optimizer
        
        self.correct_pred = tf.equal(tf.argmax(tmp,1), tf.argmax(self.Y,1))
        accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))
        self.accuracy = accuracy
        
        #self.init = tf.global_variables_initializer()   
        self.init = tf.initialize_all_variables()
    def fit(self,trainX, trainY, dropout = 0.3, seed=True):
        self.train(trainX, trainY, seed=True)
        sess = self.sess
        sess.run(self.init)
        batch_size = self.batch_size
        trainY = self.dense_to_one_hot(trainY)
        for ep in range(self.training_epoch):
            for i in range(int(len(trainX)/batch_size)+1):
                if i < int(len(trainX)/batch_size)+1:
                    batch_x = trainX[i*batch_size : (i+1)*batch_size]
                    batch_y = trainY[i*batch_size : (i+1)*batch_size]
                elif i== int(len(trainX)/batch_size)+1:
                    batch_x = trainX[-batch_size:]
                    batch_y = trainY[-batch_size:]
                sess.run(self.optimizer, feed_dict={self.X:batch_x, self.Y:batch_y, self.keep_prob:(1.-dropout)})
            if ep%self.display_step==0:                
                loss, acc = sess.run([self.cost,self.accuracy], feed_dict={self.X:trainX, self.Y:trainY, self.keep_prob:1.})
                print (str(ep)+"th "+'Epoch Loss = {:.5f}'.format(loss)+" Training Accuracy={:.5f}".format(acc))
        self.sess= sess
        print("Optimization Finished!") 
    
    def pred_prob(self, testX):
        sess = self.sess
        batch_size = self.batch_size
        trainX = testX
        predict_output = np.zeros([1,self.out_classes])
        for i in range(int(len(trainX)/batch_size)+1):
            if i < int(len(trainX)/batch_size)+1:
                batch_x = trainX[i*batch_size : (i+1)*batch_size]
                batch_y = trainY[i*batch_size : (i+1)*batch_size]
            elif i== int(len(trainX)/batch_size)+1:
                batch_x = trainX[-batch_size:]
                batch_y = trainY[-batch_size:]
            tp = sess.run(self.predict,feed_dict={self.X:batch_x, self.keep_prob:1.})
            predict_output = np.row_stack([predict_output, tp])
        predict_output = np.delete(predict_output, obj=0, axis=0)
        return predict_output
    
    def pred(self, testX):
        pred_prob = self.pred_prob(testX)
        return np.argmax(pred_prob, axis=1)

In [6]:
test=test_1()

In [10]:
test.dense_to_one_hot(np.array([1,1,2,2,3,3]))

IndexError: index 18 is out of bounds for size 18

In [11]:
labels_dense=np.array([1,1,2,2,3,3])

In [32]:
num_classes = len(np.unique(labels_dense))
raws_labels = labels_dense.shape[0]#多少列数据
index_offset = np.arange(raws_labels) * num_classes
labels_one_hot = np.zeros((raws_labels, num_classes))
labels_one_hot.flat[index_offset + labels_dense.ravel()-1] = 1

In [36]:
labels_one_hot

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [37]:
labels_one_hot.transpose([1,0,2])

ValueError: axes don't match array

In [38]:
tf.split?